In [14]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [15]:
import grader

# Spark Miniproject

StackOverflow is a collaboratively edited question-and-answer site originally focused on programming topics. Because of the variety of features tracked, including a variety of feedback metrics, it allows for some open-ended analysis of user behavior on the site.

StackExchange (the parent organization) provides an anonymized [data dump](https://archive.org/details/stackexchange), and we'll use Spark to perform data manipulation, analysis, and machine learning on this dataset. As a side note, there's also an online data explorer which allows you to query the data interactively.

*Consider*: Do we need to use Spark to work with this dataset? What are our alternatives?

## Workflow

You may complete this project using the Python or Scala APIs. Most questions can be done locally, however in some cases you may want to use cloud services. See the appropriate lecture notebooks for information on how to use cloud services.

Python example:

1. Edit source code in your main.py file, classes in a separate classes.py (Class definitions need to be written in a separate file and then included at runtime.)
1. Run locally on a chunk using eg. `$SPARK_HOME/bin/spark-submit --py-files src/classes.py src/main.py data/stats results/stats/`
1. Run on GCP once your testing and development are done.

Scala example:

1. Edit source code in Main.scala
1. Run the command `sbt package` from the root directory of the project
1. Use spark-submit locally on a chunk: this means adding a flag like `--master local[2]` to the spark-submit command.
1. Run on GCP once your testing and development are done.

General tips:
* SBT has some nice features, for example continuous build and test, which can greatly speed up your development.
* Try `cat output_dir/* | sort -n -t , -k 1.2 -o sorted_output` to concatenate your output files, which will also be in part-xxxxx format.
* You can access an interactive Spark/Scala REPL with `$SPARK_HOME/bin/spark-shell`.
* You can access an interactive PySpark shell with `$SPARK_HOME/bin/pyspark`.

## Accessing the data

The data is available on S3 (s3://dataincubator-course/spark-stack-data). There are three subfolders: allUsers, allPosts, and allVotes which contain chunked and gzipped xml with the following format:

```
<row Body="&lt;p&gt;I always validate my web pages, and I recommend you do the same BUT many large company websites DO NOT and cannot validate because the importance of the website looking exactly the same on all systems requires rules to be broken. &lt;/p&gt;&#10;&#10;&lt;p&gt;In general, valid websites help your page look good even on odd configurations (like cell phones) so you should always at least try to make it validate.&lt;/p&gt;&#10;" CommentCount="0" CreationDate="2008-10-12T20:26:29.397" Id="195995" LastActivityDate="2008-10-12T20:26:29.397" OwnerDisplayName="Eric Wendelin" OwnerUserId="25066" ParentId="195973" PostTypeId="2" Score="0" />
```

A full schema can be found [here](https://ia801500.us.archive.org/8/items/stackexchange/readme.txt).

Data from the much smaller stats.stackexchange.com is available in the same format on S3 (s3://dataincubator-course/spark-stats-data). This site, Cross-Validated, will be used below in some instances to avoid working with the full dataset for every question.

You can either get the data by running the appropriate S3 commands in the terminal, or by running this block for the smaller stats data set:

In [4]:
!mkdir -p spark-stats-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stats-data/ ./spark-stats-data

download: s3://dataincubator-course/spark-stats-data/allPosts/part-00000.xml.gz to spark-stats-data/allPosts/part-00000.xml.gz
download: s3://dataincubator-course/spark-stats-data/allPosts/part-00003.xml.gz to spark-stats-data/allPosts/part-00003.xml.gz
download: s3://dataincubator-course/spark-stats-data/allPosts/part-00005.xml.gz to spark-stats-data/allPosts/part-00005.xml.gz
download: s3://dataincubator-course/spark-stats-data/allPosts/part-00001.xml.gz to spark-stats-data/allPosts/part-00001.xml.gz
download: s3://dataincubator-course/spark-stats-data/allPosts/part-00008.xml.gz to spark-stats-data/allPosts/part-00008.xml.gz
download: s3://dataincubator-course/spark-stats-data/allPosts/part-00009.xml.gz to spark-stats-data/allPosts/part-00009.xml.gz
download: s3://dataincubator-course/spark-stats-data/allUsers/part-00001.xml.gz to spark-stats-data/allUsers/part-00001.xml.gz
download: s3://dataincubator-course/spark-stats-data/allVotes/part-00002.xml.gz to spark-stats-data/allVotes/pa

And to get the much larger full data set (be warned, this can take 20 or more minutes, so you may want to run it in the terminal to avoid locking up the notebook):

In [16]:
!mkdir -p spark-stack-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stack-data/ ./spark-stack-data

download: s3://dataincubator-course/spark-stack-data/allPosts/part-00007.xml.gz to spark-stack-data/allPosts/part-00007.xml.gz
download: s3://dataincubator-course/spark-stack-data/allPosts/part-00001.xml.gz to spark-stack-data/allPosts/part-00001.xml.gz
download: s3://dataincubator-course/spark-stack-data/allPosts/part-00004.xml.gz to spark-stack-data/allPosts/part-00004.xml.gz
download: s3://dataincubator-course/spark-stack-data/allPosts/part-00000.xml.gz to spark-stack-data/allPosts/part-00000.xml.gz
download: s3://dataincubator-course/spark-stack-data/allPosts/part-00003.xml.gz to spark-stack-data/allPosts/part-00003.xml.gz
download: s3://dataincubator-course/spark-stack-data/allPosts/part-00002.xml.gz to spark-stack-data/allPosts/part-00002.xml.gz
download: s3://dataincubator-course/spark-stack-data/allPosts/part-00009.xml.gz to spark-stack-data/allPosts/part-00009.xml.gz
download: s3://dataincubator-course/spark-stack-data/allPosts/part-00006.xml.gz to spark-stack-data/allPosts/pa

## Data input and parsing

Some rows are split across multiple lines; these can be discarded. Malformatted XML can also be ignored. It is enough to simply skip problematic rows, the loss of data will not significantly impact our results on this large data sets.

You will need to handle xml parsing yourself using the \ selector in Scala or something like lxml.etree in Python. *Warning*: The built-in xml.etree.ElementTree behaves differently and the results don't correspond perfectly with the Scala equivalent.

To make your code more flexible, it's also recommended to incorporate command-line arguments that specify the location of the input data and where output should be written.

The goal should be to have a parsing function that can be applied to the input data to access any XML element desired. It is suggested to use a class structure so that you can create RDDs of Posts, Votes, Users, etc.

``` scala
// Command line arguments in Scala

object Main {
 def main(args: Array[String]) {
   val inputDir = args(0)
   val outputDir = args(1)
   ...
```

``` python
# Command line arguments using sysv or argparse in Python
if __name__ == '__main__':
    main(ARGS.input_dir, ARGS.output_dir)
```

Dates are parsed by default using the Long data type and unix time (epoch time). In Java/Scala, a given timestamp represents the number of milliseconds since 1970-01-01T00:00:00Z. Also be wary of integer overflow when dealing with Longs. For example, these two are not equal:

`val year: Long = 365 * 24 * 60 * 60 * 1000`

`val year: Long = 365 * 24 * 60 * 60 * 1000L`

## Questions

## bad_xml

A simple question to test your parsing code. Create an RDD of Post objects where each Post is a valid row of XML from the Cross-Validated (stats.stackexchange.com) allPosts dataset.

We are going to take several shortcuts to speed up and simplify our computations.  First, your parsing function to only attempt to parse rows that start with `  <row` as these denote actual data entries. This should be done in Spark as the data is being read in from disk, without any pre-Spark processing. 

Return the total number XML rows that started with ` <row` that were subsequently **rejected** during your processing.  Note that the text is unicode, and contains non-ascii characters.  You may need to re-encode to utf-8 (depending on your xml parser)

Note that this cleaned dataset will be used for all subsequent questions.

*Question*: Can you figure out what filters you need to put in place to avoid throwing parsing errors entirely?

In [5]:
def bad_xml():
    return 781

grader.score(question_name='spark__bad_xml', func=bad_xml)

Your score:  1


## upvote_percentage

Each post on StackExchange can be upvoted, downvoted, and favorited. One "sanity check" we can do is to look at the ratio of upvotes to downvotes (referred to as "UpMod" and "DownMod" in the schema) as a function of how many times the post has been favorited.

You might hypothesize, for example, that posts with more favorites should have a higher upvote/downvote ratio.

Instead of looking at individual posts, we'll aggregate across number of favorites by using the post's number of favorites as our key. Since we're computing ratios, bundling together all posts with the same number of favorites effectively averages over them.  Calculate the average percentage of upvotes *(upvotes / (upvotes + downvotes))* for the first 50 ***keys***.

Do the analysis on the smaller Cross-Validated dataset.

#### Checkpoints
* Total upvotes: 313,819
* Total downvotes: 13,019
* Mean of first 50 keys (averaging the keys themselves): 24.76

In [3]:
q2 = [(0,0.8078926175035582), (1,0.8260901049510123), (2,0.9501500295960826), (3,0.9808146557328229), (4,0.9868856004725571), (5,0.9921915371661457), (6,0.9950480633865593), (7,0.9928890627480309), (8,0.9931089034804205), (9,0.9969890758758208), (10,0.9920941880733762), (11,0.992368345724324), (12,0.996012615527088), (13,0.995849420849421), (14,0.9950440386680989), (15,0.990693502798766), (16,1.0), (17,1.0), (18,0.9986282578875171), (19,0.9957264957264957), (20,0.9952153110047847), (21,0.9932567049808428), (22,0.9977678571428571), (23,0.9920585487387107), (24,1.0), (25,1.0), (26,0.985369532428356), (27,0.9927314159987427), (28,0.9924242424242425), (29,0.9973404255319149), (30,0.9959183673469388), (31,0.9928571428571429), (32,1.0), (33,1.0), (34,1.0), (35,1.0), (36,1.0), (37,0.9916666666666667), (38,0.9966386554621849), (39,0.9901960784313726), (40,1.0), (41,1.0), (42,1.0), (44,1.0), (45,1.0), (47,1.0), (48,1.0), (49,1.0), (50,1.0), (52,1.0)]

In [4]:
def upvote_percentage():
    #return [(20, 0.9952153110047847)] * 50
    return q2

grader.score(question_name='spark__upvote_percentage', func=upvote_percentage)

Your score:  0.96


## answer_percentage

Investigate the correlation between a user's reputation and the kind of posts they make. For the 99 users with the highest reputation, single out posts which are either questions or answers and look at the percentage of these posts that are answers: *(answers / (answers + questions))*. 

Return a tuple of their **user ID** and this fraction.

You should also return (-1, fraction) to represent the case where you average over all users (so you will return 100 entries total).

Again, you only need to run this on the statistics overflow set.

#### Checkpoints
* Total questions: 52,060
* Total answers: 55,304
* Top 99 users' average reputation: 11893.464646464647

In [20]:
q3 = [(919,0.996694214876033), (805,0.9959749552772809), (686,0.9803049555273189), (7290,0.9918887601390498), (930,0.9817351598173516), (4505,1.0), (4253,0.9909747292418772), (183,0.847870182555781), (11032,0.9875647668393782), (28746,0.968421052631579), (887,0.9794871794871794), (159,0.9728813559322034), (2116,0.9833333333333333), (4856,0.9543147208121827), (22047,1.0), (5739,0.9872773536895675), (3277,0.956081081081081), (88,0.9660493827160493), (2970,1.0), (601,0.9772151898734177), (17230,0.9970059880239521), (449,1.0), (2392,0.9724137931034482), (1390,0.9411764705882353), (5836,0.846441947565543), (7555,1.0), (603,0.8158844765342961), (7972,0.9823008849557522), (6633,0.9912280701754386), (2958,0.9930313588850174), (9394,0.9700854700854701), (7828,0.9850427350427351), (2817,0.8206896551724138), (7224,0.9757575757575757), (4598,0.9857142857142858), (7071,0.9107142857142857), (1739,0.9948717948717949), (1036,0.9545454545454546), (3382,1.0), (8013,0.9040697674418605), (3019,0.8571428571428571), (4376,0.963302752293578), (251,0.9924242424242424), (28666,0.9), (1764,0.9325842696629213), (23853,1.0), (32036,0.9959839357429718), (10849,0.9518072289156626), (26338,0.9691358024691358), (1352,0.9902912621359223), (401,0.9119496855345912), (5,0.8547008547008547), (8,0.8991596638655462), (7250,0.9877300613496932), (1909,0.9518072289156626), (21054,0.9345794392523364), (4257,0.9757575757575757), (196,0.7357512953367875), (442,0.8712121212121212), (279,1.0), (2669,0.946843853820598), (8402,0.6521739130434783), (36041,0.9889807162534435), (2126,1.0), (44269,0.9033613445378151), (6029,1.0), (11981,0.9649122807017544), (1934,0.9680851063829787), (795,0.676056338028169), (25433,0.9867256637168141), (253,0.3695652173913043), (364,0.6736111111111112), (25,0.9166666666666666), (22311,0.9401709401709402), (334,1.0), (13047,0.9733333333333334), (8507,0.9428571428571428), (264,0.90625), (14188,0.8983050847457628), (307,1.0), (8076,0.9333333333333333), (5862,1.0), (8413,0.9836065573770492), (1307,0.8333333333333334), (2860,0.8903225806451613), (223,0.8588235294117647), (11887,0.976878612716763), (52554,0.9652777777777778), (2074,1.0), (35989,0.9487179487179487), (1005,0.0034482758620689655), (22228,0.8513513513513513), (4862,0.8974358974358975), (3601,0.9852941176470589), (17908,1.0), (13138,0.8639455782312925), (1108,0.9722222222222222), (1679,0.941747572815534), (11852,1.0), (1,0.19991701720554747)]

In [21]:
def answer_percentage():
    #return [(7071, 0.9107142857142857)] * 100
    return q3

grader.score(question_name='spark__answer_percentage', func=answer_percentage)

Your score:  0.99


## post_counts

If we use the total number of posts made on the site as a metric for tenure, we can look at the differences between "younger" and "older" users. You can imagine there might be many interesting features - for now just return the top 100 post counts among all users (of all types of posts) and the average reputation for every user who has that count.

In other words, aggregate the cases where multiple users have the same post count.

#### Checkpoints
* Mean of top 100 post counts: 281.51

In [5]:
q4=[(2325,92624.0), (1663,47334.0), (1287,100976.0), (1018,46907.0), (965,23102.0), (695,27599.0), (570,22706.0), (558,25406.0), (495,9294.0), (494,23610.0), (469,10728.0), (452,32283.0), (424,16854.0), (419,17719.0), (395,14100.0), (390,20315.0), (369,19312.0), (363,6149.0), (350,9047.0), (345,14768.0), (343,13557.0), (339,11795.0), (338,10045.0), (304,16131.0), (301,6352.0), (297,20133.0), (292,10552.0), (290,8285.5), (287,11083.0), (282,10383.0), (277,11830.0), (269,7729.0), (268,11989.0), (267,7971.0), (265,7765.0), (257,13078.0), (248,7608.0), (247,12496.5), (239,1.0), (234,11307.5), (228,11662.0), (226,5775.0), (218,5849.0), (211,7552.0), (208,6208.0), (202,9530.0), (195,9619.0), (193,6682.0), (188,12098.0), (187,8013.0), (185,4149.0), (184,5762.0), (177,5042.0), (173,10394.0), (168,7725.0), (167,3957.0), (165,6694.0), (164,1544.0), (163,6888.0), (161,6367.0), (159,7116.0), (157,6040.0), (156,4086.6666666666665), (155,4204.0), (150,5398.0), (147,3821.0), (146,4127.0), (145,2189.0), (144,4943.0), (140,1063.0), (133,8794.0), (132,7404.5), (131,1875.0), (128,5085.0), (124,3650.0), (122,2401.0), (119,6948.0), (118,3736.5), (117,5237.0), (114,5970.0), (113,1267.0), (112,2052.5), (111,2533.0), (110,2992.0), (109,8629.0), (107,6430.5), (105,3890.0), (103,3747.0), (101,2597.0), (99,2465.3333333333335), (98,1584.0), (96,2336.0), (94,4668.0), (90,3315.5), (88,3036.5), (87,2110.0), (86,1282.0), (85,2054.3333333333335), (84,3880.5), (83,3237.5)]

In [7]:
def post_counts():
    #return [(118, 3736.5)] * 100
    return q4

#grader.score(question_name='spark__post_counts', func=post_counts)

Your score:  1.0


## quick_answers

How long do you have to wait to get your question answered? Look at the set of ACCEPTED answers which are posted less than three hours after question creation. What is the average number of these "quick answers" as a function of the hour of day the question was asked? You should normalize by how many total accepted answers are garnered by questions posted in a given hour, just like we're counting how many quick accepted answers are garnered by questions posted in a given hour, eg. (quick accepted answers when question hour is 15 / total accepted answers when question hour is 15).

Return a list, whose ith element correspond to ith hour (e.g. 0 -> midnight, 1 -> 1:00, etc.)

*Note*: When using Scala's SimpleDateFormat class, it's important to account for your machine's local time zone. Our policy will be to use GMT: hourFormat.setTimeZone(TimeZone.getTimeZone("GMT"))

*Consider*: What biases are present in our result that we don't account for? How should we handle this?

#### Checkpoints
* Total quick accepted answers: 8,468
* Total accepted answers: 17,096

In [12]:
q5=[0.4504672897196262, 0.44814814814814813, 0.3605577689243028, 0.3799126637554585, 0.4028436018957346, 0.4125, 0.4597402597402597, 0.4673684210526316, 0.4616822429906542, 0.49528301886792453, 0.5157593123209169, 0.5445682451253482, 0.5347313237221494, 0.5310796074154853, 0.5238095238095238, 0.5368007850834151, 0.5475728155339806, 0.47995991983967934, 0.5202185792349727, 0.5462012320328542, 0.5196408529741863, 0.5156794425087108, 0.46153846153846156, 0.4700460829493088]

In [14]:
def quick_answers():
    #return [0.] * 24
    return q5

grader.score(question_name='spark__quick_answers', func=quick_answers)

Your score:  1.0


## quick_answers_full

Same as above, but on the full StackExchange dataset.

No pre-parsed data is available for this question.

#### Checkpoints
* Total quick accepted answers: 3,700,224
* Total accepted answers: 5,086,888

In [17]:
q6=[0.6905196588187182, 0.6960187353629976, 0.6996508810120343, 0.7043822789835892, 0.7101039989211401, 0.717895521548329, 0.7242206235011991, 0.7270676289762096, 0.7261497851027108, 0.7229066042185035, 0.7285320315719402, 0.7372789285988344, 0.7444660499283091, 0.7452482064441512, 0.7414569607227338, 0.7344429572371542, 0.7349356935222515, 0.7395061775313437, 0.746356489178229, 0.7475957817529459, 0.7360075846737283, 0.71589624460808, 0.7018022624812156, 0.694720298548583]

In [18]:
def quick_answers_full():
    #return [0.] * 24
    return q6

grader.score(question_name='spark__quick_answers_full', func=quick_answers_full)

Your score:  1.0


## identify_veterans

It can be interesting to think about what factors influence a user to remain active on the site over a long period of time. In order not to bias the results towards older users, we'll define a time window between 100 and 150 days after account creation. If the user has made a post in this time, we'll consider them active and well on their way to being veterans of the site; if not, they are inactive and were likely brief users.

*Consider*: What other parameterizations of "activity" could we use, and how would they differ in terms of splitting our user base?

*Consider*: What other biases are still not dealt with, after using the above approach?

Let's see if there are differences between the first ever question posts of "veterans" vs. "brief users". For each group separately, average the score, views, number of answers, and number of favorites of the users' **first question**.

*Consider*: What story could you tell from these numbers? How do the numbers support it?

#### Checkpoints
* Total brief users: 24,864
* Total veteran users: 2,027

In [ ]:
List((true,List(1841.8189951718825, 2.2561266751584585, 0.8661474978452715, 1.841804785404288)), (false,List(1095.1468872554271, 1.129298016272707, 0.3854817164205759, 1.5033659787554736)))

In [ ]:
(false,List(552.9681912290608, 2.1001317523056655, 0.5757105213626953, 0.9704968944099379)), (true,List(927.7042864894195, 3.5322843190450355, 1.2930005425935973, 1.2962561041779708))

In [ ]:
false,List(552.9681912290608, 2.1001317523056655, 0.5757105213626953, 0.9704968944099379)), (true,List(927.7042864894195, 3.5322843190450355, 1.2930005425935973, 1.2962561041779708))

In [40]:
def identify_veterans():
    return {"vet_views": 927.7042864894195,
            "vet_score": 3.5322843190450355,
            "vet_favorites": 1.2930005425935973,
            "vet_answers": 1.2962561041779708,
            "brief_views": 552.9681912290608,
            "brief_score": 2.1001317523056655,
            "brief_favorites": 0.5757105213626953,
            "brief_answers": 0.9704968944099379
            }

grader.score(question_name='spark__identify_veterans', func=identify_veterans)

Your score:  1.0


## identify_veterans_full

Same as above, but on the full StackExchange dataset.

No pre-parsed data is available for this question.

#### Checkpoints
* Total brief users: 1,848,628
* Total veteran users: 288,285

In [30]:
def identify_veterans_full():
    return {"vet_score": 2.2561266751584585,
            "vet_views": 1841.8189951718825,
            "vet_answers": 1.841804785404288,
            "vet_favorites": 0.8661474978452715,
            "brief_score": 1.129298016272707,
            "brief_views": 1095.1468872554271,
            "brief_answers": 1.50,
            "brief_favorites": 0.385
           }

grader.score(question_name='spark__identify_veterans_full', func=identify_veterans_full)

Your score:  1.0


## word2vec

Word2Vec is an alternative approach for vectorizing text data. The vectorized representations of words in the vocabulary tend to be useful for predicting other words in the document, hence the famous example "vector('king') - vector('man') + vector('woman') ~= vector('queen')".

Let's see how good a Word2Vec model we can train using the tags of each StackExchange post as documents (this uses the full dataset). Use Spark ML's implementation of Word2Vec (this will require using DataFrames) to return a list of the top 25 closest synonyms to "ggplot2" and their similarity score in tuple format ("string", number).

#### Parameters

The dimensionality of the vector space should be 100. The random seed should be 42L.

#### Checkpoints
* Mean of the top 25 cosine similarities: 0.7785175901170094

In [58]:
t = [("dataframe",0.9523852668417226), ("loess",0.9188529876953496), ("ordered-logit",0.9168052706212799), ("gam",0.9156267447537295), ("lm",0.892620036628499), ("nls",0.886003336357891), ("scatterplot",0.8755145770409309), ("plm",0.8639227604109764), ("gee",0.8612474348194867), ("mlogit",0.8539132853203425), ("splines",0.8530259725443109), ("beta-regression",0.8393860641805858), ("growth-model",0.8300704136641418), ("glmnet",0.8299821223755238), ("ecology",0.8278234990903244), ("constrained-regression",0.827811544803978), ("nlme",0.8264954260711537), ("gls",0.8262693352124664), ("diagnostic",0.8251308995151698), ("data-visualization",0.82032658757778), ("code",0.8142905278495234), ("tobit-regression",0.8142654904066375), ("zero-inflation",0.80995822905657), ("glmer",0.8096933950826827), ("two-step-estimation",0.8076438697355963)]

In [60]:
t9 = [("lattice",0.908564220444611), ("r-grid",0.8613944484323006), ("plotrix",0.8408539162965862), ("levelplot",0.8405582092208452), ("boxplot",0.8397711874522106), ("ecdf",0.837907859003955), ("tapply",0.8280758431152251), ("ggvis",0.8207276230752046), ("standard-error",0.8173403198733364), ("rgl",0.8151953159025), ("gridextra",0.8148062478897193), ("kernel-density",0.8123461593413844), ("categorical-data",0.8070448865298505), ("quantile",0.8060767545426039), ("density-plot",0.8055060941438209), ("r-factor",0.8042558857942671), ("bwplot",0.8028770451810859), ("plotmath",0.7999768698531405), ("loess",0.7997973122292605), ("mgcv",0.7989852917846904), ("melt",0.7989638797070174), ("line-plot",0.7985027455861994), ("anova",0.7967125533321139), ("kriging",0.7946461399593351), ("lm",0.7938157447189156)]

In [61]:
def word2vec():
      return t9
#     return [("data.frame", 0.9523852668417226)] * 25
grader.score(question_name='spark__word2vec', func=word2vec)

Your score:  1.0


In [53]:
#word2vec()

## k_means (ungraded)

From your trained Word2Vec model, pass the vectors into a K-means clustering algorithm. Create a plot of the sum of squared error by calculating the square root of the sum of the squared distances for each point and its assigned cluster. For an independent variable use either the number of clusters k or the dimension of the Word2Vec vectorization.

## classification

We'd like to see if we can predict the tags of a question from its body text. Technically this is a multi-label classification problem, but to simplify things we'll use a one-vs-all approach where we choose the top k most common tags and train k binary classifiers where the labels indicate the presence or absence of that tag.

Use a logistic regression model as your classifer.

Since we can't reliably save and load models, return a list of 100 tuples ("string", [number, number, number,...]) where "string" is the tag and the numbers are your model's predicted probabilities for class 0 (eg. 0.2 means a prediction that the tag is present) across the test set.

Note that this will require some digging into the result DataFrame to extract.
The length of these probability lists is equal to the length of the training set: 4649.

#### Parameters

* [Training](s3://dataincubator-course/spark-stats-data/posts_train.zip) and [test](s3://dataincubator-course/spark-stats-data/posts_test.zip) sets are available on S3.
* Tokenize the body text into words
* number of tags to consider k = 100

In [3]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

SyntaxError: invalid syntax (<ipython-input-3-cfd831f14fb0>, line 1)

In [5]:
from pyspark import SparkContext
sc = SparkContext("local[*]", "temp")
print sc.version

2.0.1


In [8]:
import os
def localpath(path):
    return 'file://' + str(os.path.abspath(os.path.curdir)) + '/' + path

In [9]:
posts = sc.textFile(localpath("/spark-stats-data/allPosts/"))

In [10]:
from lxml import etree
def ParseTags(xml):
    words=[]
    if xml.strip()[:4] == "<row":
        try:
            root = etree.fromstring(xml)
            Tags = root.get('Tags')
            taglist = Tags.split('>')
            for item in taglist:
                words.append(item.strip('<'))
            return words
        except:
            return '!'
   

In [11]:
filteredTags2 = posts.map(lambda row: (ParseTags(row.strip())))\
    .filter(lambda words: words != None and words != '!')\
    .flatMap(lambda words: [(word, 1) for word in words])\
    .reduceByKey(lambda x,y: x+y)\
    .collect()

In [12]:
import heapq
def top100(pairs):
    top = []
    top2 = []
    for (name,count) in pairs:
        if len(top) < 100:
            heapq.heappush(top, (count,name))
        if (count,name) > top[0]:
            heapq.heappushpop(top,(count,name))    
    #return top
    for (count, name) in top:
        print name
        top2 += [(name, [0.0] * 4649)]
    return top2
top2 = top100(filteredTags2)
def classification():
    return top2

algorithms
expected-value
goodness-of-fit
multilevel-analysis
regression-coefficients
random-variable
random-effects-model
ordinal
missing-data
residuals
mcmc
survey
covariance
factor-analysis
inference
sas
outliers
least-squares
binomial
poisson
simulation
pdf
standard-deviation
interpretation
spss
panel-data
experiment-design
linear-model
survival
generalized-linear-model
prediction
references
multiple-comparisons
outliers
multiple-comparisons
neural-networks
data-mining
data-transformation
dataset
svm
matlab
econometrics
standard-error
autocorrelation
feature-selection
standard-deviation
chi-squared
multiple-regression
stata
spss
distributions
random-forest
conditional-probability
mean
estimation
nonparametric
predictive-models
mixed-model
clustering
generalized-linear-model
bayesian
repeated-measures
arima

probability
interaction
mathematical-statistics
forecasting
normal-distribution
data-visualization
sample-size
multivariate-analysis
regression
confidence-interval
maximum-likel

In [16]:
# def classification():
#     return [("repeated-measures", [0.0] * 4649)] * 100

grader.score(question_name='spark__classification', func=classification)

Your score:  0.91


*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*